<a href="https://colab.research.google.com/github/shikha-aggarwal/wodehouse-generator/blob/main/gpt2_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Check GPU allocated on Google colab

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jan 28 17:53:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Step 1. Installations

In [19]:
!git clone https://github.com/huggingface/transformers

!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/datasets
!pip list | grep -E 'transformers|tokenizers'

Cloning into 'transformers'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 60874 (delta 12), reused 12 (delta 2), pack-reused 60840
Receiving objects: 100% (60874/60874), 45.64 MiB | 28.00 MiB/s, done.
Resolving deltas: 100% (42991/42991), done.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-f6jexgoy
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-f6jexgoy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.3.0.dev0-cp36-none-any.whl size=1783713 sha256=dc9446b968eb32ffaf36acb048e1f101ed8ac32126541ebbeb04c8a03f1289b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-cgq3rpqu/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Clonin

##### [Note: If you see errors during the above installation step, restarting the runtime might help.]

#### Step 2. Imports

In [ ]:
# from changed directory
# import run_language_modeling  
# from transformers.examples import run_generation

In [3]:
# standard ML imports
import torch
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

# Text processing
import json
from pathlib import Path
from glob import glob
import os
from concurrent.futures import ProcessPoolExecutor
from itertools import chain
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Step 3. Get data

In [4]:
# I am running on Colab with data stored in Google drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Text Preprocessing

##### Using sliding window of 8 sentences

In [5]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')
sent_tokenize = sentence_tokenizer.tokenize


def flatten(iterable):
    return chain.from_iterable(iterable)


def preprocess_book(book_txt):
    start_idx = book_txt.index("START OF THIS PROJECT GUTENBERG") + 100
    end_idx = book_txt.index("END OF THIS PROJECT") - 20
    txt =  book_txt[start_idx: end_idx]
    return re.sub("\s+", " ", txt)


def process_book(book_path):
    try:
        txt = preprocess_book(Path(book_path).read_text("utf-8"))
        sentences = [s for s in sent_tokenize(txt) if len(s) >= 16]
        windowed_sentences = []
        for snt in range(len(sentences)):
            windowed_sentences.append(" ".join(sentences[snt: snt + 8]))
        return windowed_sentences
    except:
        print(f"Could not parse \n{book_path}\n")
        return []

In [ ]:
# Uncomment on first run ONLY. Once you have the training file, comment it out again.

# train_data_directory = '/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/all_novels/'
# sliding_train_data = '/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/train_sliding.txt'

# books = []
# for filename in os.listdir(train_data_directory):
#   file_path = os.path.join(train_data_directory, filename)
#   books.append(file_path)

# buffer, BUFFER_SIZE = [], 100000
# with open(sliding_train_data, "w") as file:
#   for i, sentence in enumerate(flatten(process_book(f) for f in books)):
#     if len(buffer) >= BUFFER_SIZE:
#       file.write("\n".join(buffer))
#       buffer.clear()
#       print(i, end="\r")
#     buffer.append(sentence)
#   if len(buffer) > 0:
#     file.write("\n".join(buffer))
#     buffer.clear()

In [6]:
!head /content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt

uck Greif and the Online Distributed Proofreading Team INDISCRETIONS OF ARCHIE By P. G. Wodehouse It wasn't Archie's fault really. Its true he went to America and fell in love with Lucille, the daughter of a millionaire hotel proprietor and if he did marry her--well, what else was there to do? From his point of view, the whole thing was a thoroughly good egg; but Mr. Brewster, his father-in-law, thought differently, Archie had neither money nor occupation, which was distasteful in the eyes of the industrious Mr. Brewster; but the real bar was the fact that he had once adversely criticised one of his hotels. Archie does his best to heal the breach; but, being something of an ass, genus priceless, he finds it almost beyond his powers to placate “the man-eating fish” whom Providence has given him as a father-in-law P. G. Wodehouse AUTHOR OF “THE LITTLE WARRIOR,” “A DAMSEL IN DISTRESS,” “UNEASY MONEY,” ETC. NEW YORK GEORGE H. DORAN COMPANY COPYRIGHT,1921, BY GEORGE H, DORAN COMPANY COPYRIG

In [7]:
# number of lines, words, characters respectively

!wc /content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt

  170036 18024409 99800308 /content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/train_sliding.txt


In [ ]:
## TAKES LOOOONG TIME. DO NOT RUN ONCE you have a trained model handy.

# !python run_language_modeling.py \
#     --output_dir='/content/drive/My Drive/finetuned_models/wodehouse' \
#     --model_type=gpt2 \
#     --model_name_or_path=gpt2-medium \
#     --save_total_limit=5 \
#     --num_train_epochs=1.0 \
#     --do_train \
#     --evaluate_during_training \
#     --logging_steps=500 \
#     --save_steps=1500 \
#     --train_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt \
#     --do_eval \
#     --eval_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/validate.txt \
#     --per_gpu_train_batch_size=2 \
#     --per_gpu_eval_batch_size=2 \
#     --block_size=128 \
#     --gradient_accumulation_steps=5 \
#     --overwrite_output_dir # too lazy to delete previous failed run

In [16]:
!ls

drive  sample_data  transformers


In [23]:
## TAKES LOOOONG TIME. DO NOT RUN ONCE you have a trained model handy.

import os
os.chdir('/content/transformers/examples/language-modeling')

!python run_clm.py \
    --output_dir='/content/drive/My Drive/finetuned_models_2/wodehouse' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --logging_steps=500 \
    --save_steps=1500 \
    --train_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt \
    --do_eval=y \
    --validation_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/validate.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5 \
    --overwrite_output_dir # too lazy to delete previous failed run

2021-01-28 18:13:44.368248: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/28/2021 18:13:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/28/2021 18:13:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/drive/My Drive/finetuned_models_2/wodehouse, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=5, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Jan28_18-13-45_c65b84e3a59f, logging_first_step=False, logging_steps=5

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [24]:
!ls '/content/drive/My Drive/finetuned_models_2/wodehouse'

checkpoint-12000  config.json		   tokenizer_config.json
checkpoint-13500  eval_results_clm.txt	   trainer_state.json
checkpoint-15000  merges.txt		   training_args.bin
checkpoint-16500  pytorch_model.bin	   train_results.txt
checkpoint-18000  special_tokens_map.json  vocab.json


#### Helper functions

In [25]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  #args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### How is the trained model doing?

In [26]:
class DictToObject(object):

    def __init__(self, dictionary):
        def _traverse(key, element):
            if isinstance(element, dict):
                return key, DictToObject(element)
            else:
                return key, element

        objd = dict(_traverse(k, v) for k, v in dictionary.items())
        self.__dict__.update(objd)

In [27]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models_2/wodehouse/checkpoint-18000'
OUTPUT_PATH = '/content/drive/My Drive/finetuned_models_2/wodehouse/output_checkpoint_18000'

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/validate.txt",
              "/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=OUTPUT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObject(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

Running on device:  cuda


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:954: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


NameError: ignored

### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [ ]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [ ]:
def generate_wodehouse_samples(prompt):

  # You should try out other prompts as well as no prompt at all.
  PROMPT = prompt
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  n_gpu = torch.cuda.device_count()
  print("Running on device: ", device)

  args = collections.defaultdict(
    model_name_or_path=CHECKPOINT_PATH,
    output_dir=OUTPUT_PATH,
    n_gpu=n_gpu,
    mlm=False,
    device=device,
    model_type='gpt2',
    seed=42,
    stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
    temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
    k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
    p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
    repetition_penalty=None,
    length=900,  # Number of tokens to generate.
    num_return_sequences=3,  # Number of independently computed samples to generate.
  )
  args = DictToObject(dict(args))

  model = load_model(args)
  sequences = generate_samples(args, model, PROMPT)

  return sequences


In [ ]:
def print_sequences(sequences):

  for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)
  

In [ ]:
sequences = generate_wodehouse_samples("Seated with his wife at breakfast on the veranda which overlooked the rolling lawns and leafy woods of his charming Sussex home, Geoffrey Windlebird, the great financier, was enjoying the morning sun to the full. ")
print_sequences(sequences)

In [ ]:
sequences = generate_wodehouse_samples("It was in Oxford Street at the hour when women come up from the suburbs to shop; and he was standing among the dogs and commissionaires outside Selfridge’s.")
print_sequences(sequences)

10/23/2020 21:16:17 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/checkpoint-15000/config.json
10/23/2020 21:16:17 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/23/2020 21:16:

Running on device:  cuda


10/23/2020 21:16:30 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/checkpoint-15000/config.json
10/23/2020 21:16:30 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/23/2020 21:16:


====== GENERATION 0 ======
It was in Oxford Street at the hour when women come up from the suburbs to shop; and he was standing among the dogs and commissionaires outside Selfridge’s. He was feeling that strange exhilaration, the thrill which comes to those who have done well at school, the triumphal glow which leads down the aisle at the Savoy and reaches even to the children’s section of department stores.
All he had to do was to hang about and wait, and soon the other fellow would get busy and do it at him, and it would be his triumph. As for me, I would merely sit there. He had shown me hitherto nothing but a bleak future. In fact, at the present moment I rather fancy that my existence in the neighbourhood would be a trifle dull. It seemed to me that, in addition to boredom, the spectacle of Baxter would increase the natural tenderness with which the thing happened to him. It was in Oxford Street at the hour when women come up from the suburbs to shop; and he was standing among th